<a href="https://colab.research.google.com/github/trfrancisco/PStr_P1/blob/main/ps2024_tp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processamento de Streams 2024
## TP1 - Energy Meter Monitoring


The sensor data corresponds to (periodic) readings from 11 residential energy meters. The data covers the month of February 2024, and is streamed off Kafka.

Each data sample has the following schema:

timestamp | sensor_id | energy
----------|-------------|-----------
timestamp | string  | float

Each energy value (KWh) corresponds to the accumulated value of the meter at the time of measurement. As such,
each meter is expected to produce a monotonically increasing series of pairs of timestamp and energy consummed up to that moment.

The meters do not start at zero or at the same value.

The contracted energy provider is [SU Eletricidade](https://sueletricidade.pt/en/home)

The cost of energy varies depending on the time of day, according to the table below:

vazio | super-vazio | cheias | ponta |
------|-------------|--------|-------|
0.1072€| 0.1072€ | 0.1741€ | 0.2400€|

The plan corresponds to the [daily schedule tariff](https://sueletricidade.pt/en/schedules/546/daily-and-weekly-timetable), so the schedule is the same
for all days of the week.

## Questions

For each sensor, separately:

1. Compute the running total energy consumed so far, for the month. The value should be updated every 5 minutes. (Sorted in descending order by value and sensor.)

2. Compute the running total energy consumed so far, for the day. The value should be updated every 5 minutes. (Sorted in descending order by value and sensor.)

3. For the current day, compute the total energy used in each half hour period. The value should be updated every 5 minutes. (Sorted by period; a column for each sensor)

4. Compute the running total expense for the day. The value should be updated every minute. (Sorted in descending order by value and sensor.)



## Requeriments

Solve each question using Structured Spark Streaming.

## Other Grading Criteria

+ Grading will also take into account the general clarity of the programming and of the presentation report (notebook).




### Deadline

26th April + 1/2 day - ***no penalty***

For each day late, ***0.5 / day penalty***. Penalty accumulates until the grade of the assignment reaches 8.0.

---
### Colab Setup


In [ ]:
#@title Mount Google Drive (Optional)
from google.colab import drive
drive.mount('/content/drive')

In [1]:
#@title Install PySpark
!pip install pyspark findspark --quiet
import findspark
findspark.init()
findspark.find()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


'/usr/local/lib/python3.10/dist-packages/pyspark'

In [2]:
#@title Install & Launch Kafka
%%bash
KAFKA_VERSION=3.7.0
KAFKA=kafka_2.12-$KAFKA_VERSION
wget -q -O /tmp/$KAFKA.tgz https://dlcdn.apache.org/kafka/$KAFKA_VERSION/$KAFKA.tgz
tar xfz /tmp/$KAFKA.tgz
wget -q -O $KAFKA/config/server1.properties - https://github.com/smduarte/ps2024/raw/main/colab/server1.properties

UUID=`$KAFKA/bin/kafka-storage.sh random-uuid`
$KAFKA/bin/kafka-storage.sh format -t $UUID -c $KAFKA/config/server1.properties
$KAFKA/bin/kafka-server-start.sh -daemon $KAFKA/config/server1.properties

metaPropertiesEnsemble=MetaPropertiesEnsemble(metadataLogDir=Optional.empty, dirs={/tmp/kraft-combined-logs: EMPTY})
Formatting /tmp/kraft-combined-logs with metadata.version 3.7-IV4.


### Energy sensor data publisher
This a small python Kafka client that publishes a continous stream of text lines, obtained from the periodic output of the sensors.

* The Kafka server is accessible @localhost:9092
* The events are published to the `energy` topic
* Events are published 60x faster than realtime relative to the timestamp


In [3]:
#@title Start Kafka Publisher
%%bash
pip install kafka-python dataclasses --quiet
wget -q -O - https://github.com/smduarte/ps2024/raw/main/colab/kafka-tp1-logsender.tgz | tar xfz - 2> /dev/null
wget -q -O data-sorted.csv https://github.com/smduarte/ps2024/raw/main/tp1/data-sorted.csv

nohup python kafka-tp1-logsender/publisher.py --filename data-sorted.csv --topic energy  --speedup 60 2> kafka-publisher-error.log > kafka-publisher-out.log &

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 4.6 MB/s eta 0:00:00


In [4]:
#@title Python Kafka client (For Debugging)
!pip -q install confluent-kafka
from confluent_kafka import Consumer

conf = {'bootstrap.servers': 'localhost:9092',
        'group.id': '*',
        'enable.auto.commit': False,
        'auto.offset.reset': 'earliest'}

try:
  consumer = Consumer(conf)
  consumer.subscribe(['energy'])

  while True:
    msg = consumer.poll(timeout=1.0)
    if msg is None: continue
    print(msg.value())
finally:
  consumer.close()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 12.6 MB/s eta 0:00:00
b'{"timestamp": "2024-02-01 00:00:00", "sensor_id": "D", "energy": 2615.0}'
b'{"timestamp": "2024-02-01 00:00:18", "sensor_id": "C", "energy": 1098.8}'
b'{"timestamp": "2024-02-01 00:00:25", "sensor_id": "A", "energy": 650.5}'
b'{"timestamp": "2024-02-01 00:00:33", "sensor_id": "J", "energy": 966.7}'
b'{"timestamp": "2024-02-01 00:00:42", "sensor_id": "H", "energy": 2145.4}'
b'{"timestamp": "2024-02-01 00:00:54", "sensor_id": "E", "energy": 1874.0}'
b'{"timestamp": "2024-02-01 00:01:52", "sensor_id": "K", "energy": 841.2}'
b'{"timestamp": "2024-02-01 00:02:00", "sensor_id": "E", "energy": 1874.1}'
b'{"timestamp": "2024-02-01 00:02:20", "sensor_id": "I", "energy": 927.2}'
b'{"timestamp": "2024-02-01 00:02:36", "sensor_id": "K", "energy": 841.3}'
b'{"timestamp": "2024-02-01 00:03:24", "sensor_id": "G", "energy": 833.7}'
b'{"timestamp": "2024-02-01 00:03:32", "sensor_id": "B", "energy": 627.5}'
b'{"timestamp": 

KeyboardInterrupt: 

The python code below shows the basics needed to process JSON data from Kafka source using PySpark.

Spark Streaming python documentation is found [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.streaming.html)

---
#### PySpark Kafka Stream Example


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

def dumpBatchDF(df, epoch_id):
    df.show(20, False)

spark = SparkSession \
    .builder \
    .appName('Kafka Spark Structured Streaming Example') \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1') \
    .getOrCreate()

lines = spark \
  .readStream \
  .format('kafka') \
  .option('kafka.bootstrap.servers', 'localhost:9092') \
  .option('subscribe', 'energy') \
  .option('startingOffsets', 'earliest') \
  .load() \
  .selectExpr('CAST(value AS STRING)')


schema = StructType([StructField('timestamp', TimestampType(), True),
                     StructField('sensor_id', StringType(), True),
                     StructField('energy', FloatType(), True)])

lines = lines.select( from_json(col('value'), schema).alias('data')).select('data.*')

lines = lines.groupBy(window(col("timestamp"), "5 minutes")).count()

query = lines \
    .writeStream \
    .outputMode('append') \
    .foreachBatch(dumpBatchDF) \
    .start()

query.awaitTermination(600)
query.stop()
spark.stop()

+-------------------+---------+------+
|timestamp          |sensor_id|energy|
+-------------------+---------+------+
|2024-02-01 00:00:00|D        |2615.0|
|2024-02-01 00:00:18|C        |1098.8|
|2024-02-01 00:00:25|A        |650.5 |
|2024-02-01 00:00:33|J        |966.7 |
|2024-02-01 00:00:42|H        |2145.4|
|2024-02-01 00:00:54|E        |1874.0|
|2024-02-01 00:01:52|K        |841.2 |
|2024-02-01 00:02:00|E        |1874.1|
|2024-02-01 00:02:20|I        |927.2 |
|2024-02-01 00:02:36|K        |841.3 |
|2024-02-01 00:03:24|G        |833.7 |
|2024-02-01 00:03:32|B        |627.5 |
|2024-02-01 00:04:24|D        |2615.1|
|2024-02-01 00:04:40|F        |748.0 |
|2024-02-01 00:04:44|H        |2145.5|
|2024-02-01 00:05:26|C        |1098.8|
|2024-02-01 00:05:34|A        |650.5 |
|2024-02-01 00:05:42|J        |966.7 |
|2024-02-01 00:05:46|F        |748.1 |
|2024-02-01 00:06:26|J        |966.8 |
+-------------------+---------+------+
only showing top 20 rows

+-------------------+---------+------+

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=41>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/sock

+-------------------+---------+------+
|timestamp          |sensor_id|energy|
+-------------------+---------+------+
|2024-02-01 08:34:11|K        |843.8 |
|2024-02-01 08:34:15|G        |834.5 |
+-------------------+---------+------+



Py4JError: An error occurred while calling o54.awaitTermination

### 1. Compute the running total energy consumed so far, for the month. The value should be updated every 5 minutes. (Sorted in descending order by value and sensor.)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

def dumpBatchDF(df, epoch_id):
  windows = df.select("window").distinct().orderBy("window.start").collect()



  for window in windows:

        start, end = window['window'].start, window['window'].end

        window_df = df.filter((col("window.start") == start) & (col("window.end") == end)) \
                      .orderBy(col('max_value').desc())

        window_df.show(truncate=False)

spark = SparkSession \
    .builder \
    .appName('Kafka Spark Structured Streaming Example') \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1') \
    .getOrCreate()


lines = spark \
  .readStream \
  .format('kafka') \
  .option('kafka.bootstrap.servers', 'localhost:9092') \
  .option('subscribe', 'energy') \
  .option('startingOffsets', 'earliest') \
  .load() \
  .selectExpr('CAST(value AS STRING)')


schema = StructType([StructField('timestamp', TimestampType(), True),
                     StructField('sensor_id', StringType(), True),
                     StructField('energy', FloatType(), True)])

lines = lines.select( from_json(col('value'), schema).alias('data')).select('data.*')


lines = lines.withWatermark('timestamp', '1 seconds') \
    .groupBy(window(col("timestamp"), "5 minutes", "5 minutes"), 'sensor_id').agg(max("energy").alias("max_value")) \




query = lines \
    .writeStream \
    .outputMode('append') \
    .trigger(processingTime='1 seconds') \
    .foreachBatch(dumpBatchDF) \
    .start()

query.awaitTermination(180)
query.stop()
spark.stop()

+------------------------------------------+---------+---------+
|window                                    |sensor_id|max_value|
+------------------------------------------+---------+---------+
|{2024-02-01 00:00:00, 2024-02-01 00:05:00}|D        |2615.1   |
|{2024-02-01 00:00:00, 2024-02-01 00:05:00}|H        |2145.5   |
|{2024-02-01 00:00:00, 2024-02-01 00:05:00}|E        |1874.1   |
|{2024-02-01 00:00:00, 2024-02-01 00:05:00}|C        |1098.8   |
|{2024-02-01 00:00:00, 2024-02-01 00:05:00}|J        |966.7    |
|{2024-02-01 00:00:00, 2024-02-01 00:05:00}|I        |927.2    |
|{2024-02-01 00:00:00, 2024-02-01 00:05:00}|K        |841.3    |
|{2024-02-01 00:00:00, 2024-02-01 00:05:00}|G        |833.7    |
|{2024-02-01 00:00:00, 2024-02-01 00:05:00}|F        |748.0    |
|{2024-02-01 00:00:00, 2024-02-01 00:05:00}|A        |650.5    |
|{2024-02-01 00:00:00, 2024-02-01 00:05:00}|B        |627.5    |
+------------------------------------------+---------+---------+

+-----------------------

ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 120, in call
    raise e
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "<ipython-input-1-5b3e292169a0>", line 20, in dumpBatchDF
    window_df.show(truncate=False)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py", line 945, in show
    print(self._show_string(n, truncate, vertical))
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py", line 976, in _show_string
    return self._jdf.showString(n, int_truncate, vertical)
  File "/usr/local/lib/python3.10/dist-packages/py